In [ ]:
import datetime
import math
import os
import sys
from collections import defaultdict
from typing import Union

import dateutil.parser
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from benthicnet.utils import sanitize_filename, sanitize_filename_series
from IPython.display import display
from tqdm.auto import tqdm

sys.path.append("..")
from pangaea_downloader.tools import checker

In [ ]:
dirname = "../query-outputs_2022-01-01"

## Load datasets and check distribution of column frequencies

In [ ]:
def get_dataset_url(ds_id: Union[str, int]) -> str:
    """Return dataset URL given six digit dataset ID."""
    if isinstance(ds_id, int):
        ds_id = str(ds_id)
    if ds_id.startswith("pangaea"):
        ds_id = ds_id.split("-")[-1]
    return f"https://doi.pangaea.de/10.1594/PANGAEA.{ds_id}"

In [ ]:
def find_url_column(df: pd.DataFrame) -> pd.Series:
    """Find and return column with image URLs."""
    # Standardize column names
    clean_cols = [
        col.lower().replace(" ", "").replace("-", "").replace("_", "").replace(".", "")
        for col in df.columns
    ]
    # Ordered list of priorities
    # Exclude url meta/ref/source which are not links to images
    candidates = [
        "urlimage",
        "urlraw",
        "urlfile",
        "url",
        "urlgraphic",
        "urlthumb",
        "urlthumbnail",
        "image",
        "imagery",
    ]
    # Find and return the first match
    for candidate in candidates:
        if candidate not in clean_cols:
            continue
        col = df.columns[clean_cols.index(candidate)]
        if any(df[col].apply(checker.is_url)):
            return col

In [ ]:
# Column name value counts
column_count = defaultdict(lambda: 0)
# Key = column name : value = dataframes with that column
column_examples = defaultdict(lambda: [])

# Files with URL issues
files_without_url = []
files_with_repeat_urls = []

# Counts
n_total = 0
n_valid = 0

verbose = False

for fname in tqdm(os.listdir(dirname)):
    # Load dataset
    ds_id = os.path.splitext(fname)[0]
    f_path = os.path.join(dirname, fname)
    df = pd.read_csv(f_path, low_memory=False)
    n_total += 1

    # Any column names with URL or Image?
    if not checker.has_url_col(df):
        continue
    # Extract the column name
    url_col = find_url_column(df)

    # No URL column found
    if not url_col:
        if verbose:
            print(f"No url column for {fname} with columns\n{df.columns}")
        files_without_url.append(fname)
        continue

    # URL column found!
    n_valid += 1
    for col in df.columns:
        col = col.lower().strip()
        column_count[col] += 1
        column_examples[col].append(fname)
    subdf = df[df[url_col] != ""]
    if len(subdf) != len(subdf.drop_duplicates(subset=url_col)):
        files_with_repeat_urls.append(fname)

In [ ]:
print(f"There are {n_valid} valid (of {n_total}) total datasets")
print(f"Of which {len(files_with_repeat_urls)} have repeated URLs", end="")
print(" (possibly multiple annotations)\n")
print(f"There are {len(column_count)} unique column names:\n")

# Sort by value in descending order
sorted_column_count = dict(
    sorted(column_count.items(), key=lambda item: item[1], reverse=True)
)
for col, count in sorted_column_count.items():
    c = col + " "
    print(f"{c:.<35s} {count:4d}")

### Examine each of the columns of interest
- Elevation
- Depth water
- Altitude

#### Elevation
**Observations:**
- ***Elevation*** values in MOST datasets are negative.
- Therefore it is reasonable to assume that ***elevation*** represents the distance of the seafloor from mean sea level.
- Source may be onboard sensors or previously recorded seafloor elevation data.
- There are a few expections (`val_expections`)
    - Positive and zero elevation values dont make sense for underwater photographs
    - These values were probably scraped from the dataset webpage

In [ ]:
# Column to find
key = "elevation"

val_expections = {}
for i, file in enumerate(column_examples[key]):
    df = pd.read_csv(os.path.join(dirname, file))
    url_column = find_url_column(df)
    df.columns = [col.lower() for col in df.columns]
    # Extract info
    mean = df[key].mean()
    sd = df[key].std()
    min_ = df[key].min()
    max_ = df[key].max()
    url = get_dataset_url(file)
    # Show
    print(f"[{i}] Mean: {mean:.2f} ± {sd:.2f} Range: {min_:.2f} to {max_:.2f}")
    # Datasets that defy column value norms
    if not ((min_ < 0) and (max_ < 0)):
        val_expections[url] = (mean, sd, min_, max_)

In [ ]:
val_expections

#### Depth water
**Observations:**
- ***Depth water*** values in ALL datasets are positive.
- Therefore it is reasonable to assume that ***depth water*** represents the absolute distance of the camera vehicle below mean sea level.

In [ ]:
# Column to find
key = "depth water"

val_expections = {}
for i, file in enumerate(column_examples[key]):
    df = pd.read_csv(os.path.join(dirname, file))
    url_column = find_url_column(df)
    df.columns = [col.lower() for col in df.columns]
    # Extract info
    mean = df[key].mean()
    sd = df[key].std()
    min_ = df[key].min()
    max_ = df[key].max()
    url = get_dataset_url(file)
    # Show
    print(f"[{i}] Mean: {mean:.2f} ± {sd:.2f} Range: {min_:.2f} to {max_:.2f}")
    # Datasets that defy column value norms
    if (min_ < 0) and (max_ < 0):
        val_expections[url] = (mean, sd, min_, max_)

In [ ]:
val_expections

#### Altitude
**Observations:**
- ***Altitude*** values in MOST (4 of 5) datasets are positive.
- Only exception: values negative (probably same as *elevation*).

In [ ]:
# Column to find
key = "altitude"

for i, file in enumerate(column_examples[key]):
    df = pd.read_csv(os.path.join(dirname, file))
    url_column = find_url_column(df)
    df.columns = [col.lower() for col in df.columns]
    # Extract info
    mean = df[key].mean()
    sd = df[key].std()
    min_ = df[key].min()
    max_ = df[key].max()
    url = get_dataset_url(file)
    # Show
    print(f"[{i}] Mean: {mean:.2f} ± {sd:.2f} Range: {min_:.2f} to {max_:.2f}")
    plt.figure(figsize=(6, 4))
    plt.plot(-df["altitude"], label="altitude")
    plt.show()

## Plot `depth water` and `elevation` values from largest dataset

In [ ]:
# Load up largest dataset, containing long cruises
df = pd.read_csv(os.path.join(dirname, "882349.csv"))
print(df.columns)
url_column = find_url_column(df)
print("Sample URL:", df[url_column].iloc[0])
print("Dataset Title:", df["dataset_title"].iloc[0])
print(df.doi.iloc[0])

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(-df["Depth water"], label="Depth water")
plt.plot(df["Elevation"], label="Elevation")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(df["Depth water"] + df["Elevation"])
plt.show()

## Datasets with both `depth water` and `elevation` columns

In [ ]:
elevation_set = set(column_examples["elevation"])
depth_water_set = set(column_examples["depth water"])
intersect = elevation_set.intersection(depth_water_set)

print("elevation_set :", len(elevation_set))
print("depth_water_set :", len(depth_water_set))
print("# of files with both:", len(intersect))

for file in intersect:
    df = pd.read_csv(os.path.join(dirname, file))
    plt.figure(figsize=(6, 4))
    plt.plot(-df["Depth water"], label="Depth water")
    plt.plot(df["Elevation"], label="Elevation")
    plt.legend()
    plt.show()

## Datasets with both `altitude` and `elevation` columns

In [ ]:
elevation_set = set(column_examples["elevation"])
altitude_set = set(column_examples["altitude"])
intersect = elevation_set.intersection(altitude_set)

print("elevation_set :", len(elevation_set))
print("altitude_set :", len(altitude_set))
print("# of files with both:", len(intersect))

for file in intersect:
    df = pd.read_csv(os.path.join(dirname, file))
    df.columns = [col.lower() for col in df.columns]
    print("All values same:", all(df.altitude == df.elevation))
    plt.figure(figsize=(6, 4))
    plt.plot(df["altitude"], label="Altitude")
    plt.plot(df["elevation"], label="Elevation")
    plt.legend()
    plt.show()

In [ ]:
df[df.altitude != df.elevation][["altitude", "elevation"]]

## Datasets with `altitude`, `depth water` and `elevation` columns

In [ ]:
elevation_set = set(column_examples["elevation"])
depth_water_set = set(column_examples["depth water"])
altitude_set = set(column_examples["altitude"])
intersect = elevation_set.intersection(depth_water_set).intersection(altitude_set)
intersect